In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score 
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

/Users/nirjharagarwal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data = pd.read_csv("cancer_before_and_after_Chernobyl_Belarus.csv")
data

,year,sex,location,cancer_type,cases
0,1985,women,Gomel,skin,17.3
1,1985,women,Mogilev,skin,16.8
2,1986,women,Gomel,skin,15.6
3,1986,women,Mogilev,skin,18.3
4,1985,women,Belarus,skin,14.9
...,...,...,...,...,...
79,1985,men,Mogilev,kidney,3.1
80,1986,men,Gomel,kidney,6.3
81,1986,men,Mogilev,kidney,6.8
82,1985,men,Belarus,kidney,3.5


In [29]:
data['cancer_type'].unique()

array(['skin', 'thyroid', 'lungs', 'pancreas', 'colon', 'rectum',
       'kidney'], dtype=object)

In [30]:
data.isnull().sum()

year           0
sex            0
location       0
cancer_type    0
cases          0
dtype: int64

In [31]:
data.describe()

,year,cases
count,84.000000,84.000000
mean,1985.500000,10.688095
std,0.503003,13.924031
min,1985.000000,0.500000
25%,1985.000000,3.500000
50%,1985.500000,6.350000
75%,1986.000000,10.275000
max,1986.000000,73.100000


In [32]:
dummies = pd.get_dummies(data.sex)
dummies

,men,women
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
79,1,0
80,1,0
81,1,0
82,1,0


In [33]:
merged = pd.concat([data,dummies],axis='columns')
merged

,year,sex,location,cancer_type,cases,men,women
0,1985,women,Gomel,skin,17.3,0,1
1,1985,women,Mogilev,skin,16.8,0,1
2,1986,women,Gomel,skin,15.6,0,1
3,1986,women,Mogilev,skin,18.3,0,1
4,1985,women,Belarus,skin,14.9,0,1
...,...,...,...,...,...,...,...
79,1985,men,Mogilev,kidney,3.1,1,0
80,1986,men,Gomel,kidney,6.3,1,0
81,1986,men,Mogilev,kidney,6.8,1,0
82,1985,men,Belarus,kidney,3.5,1,0


In [35]:
final = merged.drop(['men','sex'], axis='columns')
final
data = final
data

,year,location,cancer_type,cases,women
0,1985,Gomel,skin,17.3,1
1,1985,Mogilev,skin,16.8,1
2,1986,Gomel,skin,15.6,1
3,1986,Mogilev,skin,18.3,1
4,1985,Belarus,skin,14.9,1
...,...,...,...,...,...
79,1985,Mogilev,kidney,3.1,0
80,1986,Gomel,kidney,6.3,0
81,1986,Mogilev,kidney,6.8,0
82,1985,Belarus,kidney,3.5,0


In [36]:
dummies = pd.get_dummies(data.location)
dummies

,Belarus,Gomel,Mogilev
0,0,1,0
1,0,0,1
2,0,1,0
3,0,0,1
4,1,0,0
...,...,...,...
79,0,0,1
80,0,1,0
81,0,0,1
82,1,0,0


In [37]:
merged = pd.concat([data,dummies],axis='columns')
merged

,year,location,cancer_type,cases,women,Belarus,Gomel,Mogilev
0,1985,Gomel,skin,17.3,1,0,1,0
1,1985,Mogilev,skin,16.8,1,0,0,1
2,1986,Gomel,skin,15.6,1,0,1,0
3,1986,Mogilev,skin,18.3,1,0,0,1
4,1985,Belarus,skin,14.9,1,1,0,0
...,...,...,...,...,...,...,...,...
79,1985,Mogilev,kidney,3.1,0,0,0,1
80,1986,Gomel,kidney,6.3,0,0,1,0
81,1986,Mogilev,kidney,6.8,0,0,0,1
82,1985,Belarus,kidney,3.5,0,1,0,0


In [38]:
final = merged.drop(['Mogilev','location'], axis='columns')
final
data = final
data

,year,cancer_type,cases,women,Belarus,Gomel
0,1985,skin,17.3,1,0,1
1,1985,skin,16.8,1,0,0
2,1986,skin,15.6,1,0,1
3,1986,skin,18.3,1,0,0
4,1985,skin,14.9,1,1,0
...,...,...,...,...,...,...
79,1985,kidney,3.1,0,0,0
80,1986,kidney,6.3,0,0,1
81,1986,kidney,6.8,0,0,0
82,1985,kidney,3.5,0,1,0


In [39]:
dummies = pd.get_dummies(data.cancer_type)
dummies

,colon,kidney,lungs,pancreas,rectum,skin,thyroid
0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
79,0,1,0,0,0,0,0
80,0,1,0,0,0,0,0
81,0,1,0,0,0,0,0
82,0,1,0,0,0,0,0


In [40]:
merged = pd.concat([data,dummies],axis='columns')
merged

,year,cancer_type,cases,women,Belarus,Gomel,colon,kidney,lungs,pancreas,rectum,skin,thyroid
0,1985,skin,17.3,1,0,1,0,0,0,0,0,1,0
1,1985,skin,16.8,1,0,0,0,0,0,0,0,1,0
2,1986,skin,15.6,1,0,1,0,0,0,0,0,1,0
3,1986,skin,18.3,1,0,0,0,0,0,0,0,1,0
4,1985,skin,14.9,1,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1985,kidney,3.1,0,0,0,0,1,0,0,0,0,0
80,1986,kidney,6.3,0,0,1,0,1,0,0,0,0,0
81,1986,kidney,6.8,0,0,0,0,1,0,0,0,0,0
82,1985,kidney,3.5,0,1,0,0,1,0,0,0,0,0


In [41]:
final = merged.drop(['colon','cancer_type'], axis='columns')
final
data = final
data

,year,cases,women,Belarus,Gomel,kidney,lungs,pancreas,rectum,skin,thyroid
0,1985,17.3,1,0,1,0,0,0,0,1,0
1,1985,16.8,1,0,0,0,0,0,0,1,0
2,1986,15.6,1,0,1,0,0,0,0,1,0
3,1986,18.3,1,0,0,0,0,0,0,1,0
4,1985,14.9,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
79,1985,3.1,0,0,0,1,0,0,0,0,0
80,1986,6.3,0,0,1,1,0,0,0,0,0
81,1986,6.8,0,0,0,1,0,0,0,0,0
82,1985,3.5,0,1,0,1,0,0,0,0,0


In [43]:
dummies = pd.get_dummies(data.year)
dummies

,1985,1986
0,1,0
1,1,0
2,0,1
3,0,1
4,1,0
...,...,...
79,1,0
80,0,1
81,0,1
82,1,0


In [44]:
merged = pd.concat([data,dummies],axis='columns')
merged

,year,cases,women,Belarus,Gomel,kidney,lungs,pancreas,rectum,skin,thyroid,1985,1986
0,1985,17.3,1,0,1,0,0,0,0,1,0,1,0
1,1985,16.8,1,0,0,0,0,0,0,1,0,1,0
2,1986,15.6,1,0,1,0,0,0,0,1,0,0,1
3,1986,18.3,1,0,0,0,0,0,0,1,0,0,1
4,1985,14.9,1,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1985,3.1,0,0,0,1,0,0,0,0,0,1,0
80,1986,6.3,0,0,1,1,0,0,0,0,0,0,1
81,1986,6.8,0,0,0,1,0,0,0,0,0,0,1
82,1985,3.5,0,1,0,1,0,0,0,0,0,1,0


In [46]:
final = merged.drop([1985,'year'], axis='columns')
final
data = final
data

,cases,women,Belarus,Gomel,kidney,lungs,pancreas,rectum,skin,thyroid,1986
0,17.3,1,0,1,0,0,0,0,1,0,0
1,16.8,1,0,0,0,0,0,0,1,0,0
2,15.6,1,0,1,0,0,0,0,1,0,1
3,18.3,1,0,0,0,0,0,0,1,0,1
4,14.9,1,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
79,3.1,0,0,0,1,0,0,0,0,0,0
80,6.3,0,0,1,1,0,0,0,0,0,1
81,6.8,0,0,0,1,0,0,0,0,0,1
82,3.5,0,1,0,1,0,0,0,0,0,0


In [47]:
data.describe()

,cases,women,Belarus,Gomel,kidney,lungs,pancreas,rectum,skin,thyroid,1986
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
mean,10.688095,0.500000,0.333333,0.333333,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.500000
std,13.924031,0.503003,0.474236,0.474236,0.352029,0.352029,0.352029,0.352029,0.352029,0.352029,0.503003
min,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.350000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
75%,10.275000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,73.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [48]:
X = data.drop(columns = 'cases')
y = data['cases']

In [49]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.30, random_state= 355)

In [50]:
clf = DecisionTreeRegressor( min_samples_split= 2)
clf.fit(x_train,y_train)

DecisionTreeRegressor()

In [51]:
 clf.score(x_train,y_train)

1.0

In [52]:
py_pred = clf.predict(x_test)

In [53]:
clf.score(x_test,y_test)

0.9070472363219381

In [57]:
grid_param = {
    'max_depth' : range(2,32,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'splitter' : ['best', 'random']
    
}

In [58]:
grid_search = GridSearchCV(estimator=clf,
                     param_grid=grid_param,
                     cv=5,
                    n_jobs =-1)

In [59]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']})

In [60]:
best_parameters = grid_search.best_params_
print(best_parameters)

{'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'random'}


In [61]:
grid_search.best_score_

0.899527893423176

In [62]:
clf = DecisionTreeRegressor(max_depth =11, min_samples_leaf= 1, min_samples_split= 3, splitter ='random')
clf.fit(x_train,y_train)

DecisionTreeRegressor(max_depth=11, min_samples_split=3, splitter='random')

In [63]:
clf.score(x_test,y_test)

0.9461801530955134

In [65]:
rand_clf = RandomForestRegressor(random_state=6)

In [66]:
rand_clf.fit(x_train,y_train)

RandomForestRegressor(random_state=6)

In [67]:
rand_clf.score(x_test,y_test)

0.9524144451904167

In [68]:
grid_param = {
    "n_estimators" : [90,100,115,130],
    'max_depth' : range(2,20,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'max_features' : ['auto','log2']
}

In [69]:
grid_search = GridSearchCV(estimator=rand_clf,param_grid=grid_param,cv=5,n_jobs =-1,verbose = 3)

In [70]:
grid_search.fit(x_train,y_train)

Fitting 5 folds for each of 10368 candidates, totalling 51840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 3856 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 4592 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 5392 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 6256 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | e

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=6), n_jobs=-1,
             param_grid={'max_depth': range(2, 20),
                         'max_features': ['auto', 'log2'],
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'n_estimators': [90, 100, 115, 130]},
             verbose=3)

In [71]:
grid_search.best_params_

{'max_depth': 6,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 115}

In [74]:
rand_clf = RandomForestRegressor(
 max_depth = 6,
 max_features = 'auto',
 min_samples_leaf = 1,
 min_samples_split= 3,
 n_estimators = 115,random_state=6)

In [75]:
rand_clf.fit(x_train,y_train)

RandomForestRegressor(max_depth=6, min_samples_split=3, n_estimators=115,
                      random_state=6)

In [77]:
rand_clf.score(x_test,y_test)

0.9491871060910116

In [96]:
from sklearn.neighbors import KNeighborsRegressor
#Let's start with importing necessary libraries
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [97]:
knn = KNeighborsRegressor()
knn.fit(x_train,y_train)

KNeighborsRegressor()

In [98]:
y_pred = knn.predict(x_test)

In [99]:
knn.score(x_train,y_train)

0.742706284905075

In [102]:
param_grid = { 'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
               'leaf_size' : [18,20,25,27,30,32,34],
               'n_neighbors' : [3,5,7,9,10,11,12,13]
              }

In [103]:
gridsearch = GridSearchCV(knn, param_grid,verbose=3)

In [104]:
gridsearch.fit(x_train,y_train)

Fitting 5 folds for each of 168 candidates, totalling 840 fits
[CV] algorithm=ball_tree, leaf_size=18, n_neighbors=3 ................
[CV]  algorithm=ball_tree, leaf_size=18, n_neighbors=3, score=-4.175, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=18, n_neighbors=3 ................
[CV]  algorithm=ball_tree, leaf_size=18, n_neighbors=3, score=0.912, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=18, n_neighbors=3 ................
[CV]  algorithm=ball_tree, leaf_size=18, n_neighbors=3, score=0.569, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=18, n_neighbors=3 ................
[CV]  algorithm=ball_tree, leaf_size=18, n_neighbors=3, score=0.679, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=18, n_neighbors=3 ................
[CV]  algorithm=ball_tree, leaf_size=18, n_neighbors=3, score=0.828, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=18, n_neighbors=5 ................
[CV]  algorithm=ball_tree, leaf_size=18, n_neighbors=5, score=-2.784, total=   0.0s
[CV] algori

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  algorithm=ball_tree, leaf_size=20, n_neighbors=5, score=0.444, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=20, n_neighbors=5 ................
[CV]  algorithm=ball_tree, leaf_size=20, n_neighbors=5, score=0.508, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=20, n_neighbors=7 ................
[CV]  algorithm=ball_tree, leaf_size=20, n_neighbors=7, score=-1.591, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=20, n_neighbors=7 ................
[CV]  algorithm=ball_tree, leaf_size=20, n_neighbors=7, score=0.813, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=20, n_neighbors=7 ................
[CV]  algorithm=ball_tree, leaf_size=20, n_neighbors=7, score=0.552, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=20, n_neighbors=7 ................
[CV]  algorithm=ball_tree, leaf_size=20, n_neighbors=7, score=0.458, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=20, n_neighbors=7 ................
[CV]  algorithm=ball_tree, leaf_size=20, n_neighbors=7, score=0.316, total=

[CV]  algorithm=ball_tree, leaf_size=27, n_neighbors=13, score=-2.324, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=27, n_neighbors=13 ...............
[CV]  algorithm=ball_tree, leaf_size=27, n_neighbors=13, score=0.442, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=27, n_neighbors=13 ...............
[CV]  algorithm=ball_tree, leaf_size=27, n_neighbors=13, score=0.457, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=27, n_neighbors=13 ...............
[CV]  algorithm=ball_tree, leaf_size=27, n_neighbors=13, score=0.263, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=27, n_neighbors=13 ...............
[CV]  algorithm=ball_tree, leaf_size=27, n_neighbors=13, score=0.128, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=30, n_neighbors=3 ................
[CV]  algorithm=ball_tree, leaf_size=30, n_neighbors=3, score=-3.979, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=30, n_neighbors=3 ................
[CV]  algorithm=ball_tree, leaf_size=30, n_neighbors=3, score=0.924, 

[CV]  algorithm=ball_tree, leaf_size=32, n_neighbors=7, score=-1.722, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=32, n_neighbors=7 ................
[CV]  algorithm=ball_tree, leaf_size=32, n_neighbors=7, score=0.516, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=32, n_neighbors=7 ................
[CV]  algorithm=ball_tree, leaf_size=32, n_neighbors=7, score=0.664, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=32, n_neighbors=7 ................
[CV]  algorithm=ball_tree, leaf_size=32, n_neighbors=7, score=0.332, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=32, n_neighbors=7 ................
[CV]  algorithm=ball_tree, leaf_size=32, n_neighbors=7, score=0.349, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=32, n_neighbors=9 ................
[CV]  algorithm=ball_tree, leaf_size=32, n_neighbors=9, score=-1.130, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=32, n_neighbors=9 ................
[CV]  algorithm=ball_tree, leaf_size=32, n_neighbors=9, score=0.452, total

...

[CV]  algorithm=brute, leaf_size=27, n_neighbors=5, score=0.661, total=   0.0s
[CV] algorithm=brute, leaf_size=27, n_neighbors=5 ....................
[CV]  algorithm=brute, leaf_size=27, n_neighbors=5, score=0.509, total=   0.0s
[CV] algorithm=brute, leaf_size=27, n_neighbors=7 ....................
[CV]  algorithm=brute, leaf_size=27, n_neighbors=7, score=-3.664, total=   0.0s
[CV] algorithm=brute, leaf_size=27, n_neighbors=7 ....................
[CV]  algorithm=brute, leaf_size=27, n_neighbors=7, score=0.527, total=   0.0s
[CV] algorithm=brute, leaf_size=27, n_neighbors=7 ....................
[CV]  algorithm=brute, leaf_size=27, n_neighbors=7, score=0.630, total=   0.0s
[CV] algorithm=brute, leaf_size=27, n_neighbors=7 ....................
[CV]  algorithm=brute, leaf_size=27, n_neighbors=7, score=0.462, total=   0.0s
[CV] algorithm=brute, leaf_size=27, n_neighbors=7 ....................
[CV]  algorithm=brute, leaf_size=27, n_neighbors=7, score=0.352, total=   0.0s
[CV] algorithm=brute

[CV]  algorithm=brute, leaf_size=30, n_neighbors=11, score=-3.606, total=   0.0s
[CV] algorithm=brute, leaf_size=30, n_neighbors=11 ...................
[CV]  algorithm=brute, leaf_size=30, n_neighbors=11, score=0.449, total=   0.0s
[CV] algorithm=brute, leaf_size=30, n_neighbors=11 ...................
[CV]  algorithm=brute, leaf_size=30, n_neighbors=11, score=0.252, total=   0.0s
[CV] algorithm=brute, leaf_size=30, n_neighbors=11 ...................
[CV]  algorithm=brute, leaf_size=30, n_neighbors=11, score=0.277, total=   0.0s
[CV] algorithm=brute, leaf_size=30, n_neighbors=11 ...................
[CV]  algorithm=brute, leaf_size=30, n_neighbors=11, score=0.167, total=   0.0s
[CV] algorithm=brute, leaf_size=30, n_neighbors=12 ...................
[CV]  algorithm=brute, leaf_size=30, n_neighbors=12, score=-2.925, total=   0.0s
[CV] algorithm=brute, leaf_size=30, n_neighbors=12 ...................
[CV]  algorithm=brute, leaf_size=30, n_neighbors=12, score=0.412, total=   0.0s
[CV] algorit

[CV]  algorithm=brute, leaf_size=34, n_neighbors=3, score=0.921, total=   0.0s
[CV] algorithm=brute, leaf_size=34, n_neighbors=3 ....................
[CV]  algorithm=brute, leaf_size=34, n_neighbors=3, score=0.545, total=   0.0s
[CV] algorithm=brute, leaf_size=34, n_neighbors=3 ....................
[CV]  algorithm=brute, leaf_size=34, n_neighbors=3, score=0.748, total=   0.0s
[CV] algorithm=brute, leaf_size=34, n_neighbors=3 ....................
[CV]  algorithm=brute, leaf_size=34, n_neighbors=3, score=0.838, total=   0.0s
[CV] algorithm=brute, leaf_size=34, n_neighbors=5 ....................
[CV]  algorithm=brute, leaf_size=34, n_neighbors=5, score=-3.441, total=   0.0s
[CV] algorithm=brute, leaf_size=34, n_neighbors=5 ....................
[CV]  algorithm=brute, leaf_size=34, n_neighbors=5, score=0.599, total=   0.0s
[CV] algorithm=brute, leaf_size=34, n_neighbors=5 ....................
[CV]  algorithm=brute, leaf_size=34, n_neighbors=5, score=0.698, total=   0.0s
[CV] algorithm=brute

[Parallel(n_jobs=1)]: Done 840 out of 840 | elapsed:    3.2s finished


GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [18, 20, 25, 27, 30, 32, 34],
                         'n_neighbors': [3, 5, 7, 9, 10, 11, 12, 13]},
             verbose=3)

In [105]:
gridsearch.best_params_

{'algorithm': 'ball_tree', 'leaf_size': 18, 'n_neighbors': 7}

In [107]:
knn = KNeighborsRegressor(algorithm = 'ball_tree', leaf_size =18, n_neighbors =7)

In [108]:
knn.fit(x_train,y_train)

KNeighborsRegressor(algorithm='ball_tree', leaf_size=18, n_neighbors=7)

In [109]:
knn.score(x_train,y_train)

0.6295652313272073

In [110]:
knn.score(x_test,y_test)

0.5406191985651803